In [1]:
import datetime
import pandas as pd
import sys

sys.path.append("/home/sergey/drclinics/reports")
sys.path.append("/home/sergey/drclinics/common")

from log import log
from utils import get_path, DATETIME_FORMAT
from report_utils import save_to_excel, send_mail
from connect_platform import connect_telemed

In [2]:
sql_sub = """
select 	prod.full_name product_name,
		prod.amount,
		patient.id patient_id,
		person.first_name,
		person.middle_name,
		person.last_name,
		par."name" partner,
		p.start_ + 3 * interval '1 hour' sub_start,
		p.end_ + 3 * interval '1 hour' sub_end
from promotion p
inner join product_condition pc
   on p.product_condition_id = pc.id
inner join product prod
   on prod.id = pc.product_id
---
inner join patient_promotion pp on p.id = pp.promotion_id 
inner join patient on pp.patient_id = patient.id 
inner join person on patient.person_id = person.id 
---
inner join partner par on p.partner_id = par.id
where lower(prod.full_name) like '%подписка%' 
and lower(first_name) not like '%test%'
"""

In [3]:
sql_app = """

select 	prod.full_name product_name,
		prod.amount,
		patient.id patient_id,
		person.first_name,
		person.middle_name,
		person.last_name,
		par."name" partner,
		p.start_ + 3 * interval '1 hour' sub_start,
		p.end_ + 3 * interval '1 hour' sub_end,
		a.id,
		a.finished + 3 * interval '1 hour' app_finished
from promotion p
inner join product_condition pc
   on p.product_condition_id = pc.id
inner join product prod
   on prod.id = pc.product_id
---
inner join patient_promotion pp on p.id = pp.promotion_id 
inner join patient on pp.patient_id = patient.id 
inner join person on patient.person_id = person.id 
---
inner join partner par on p.partner_id = par.id
---
full join appointment a on patient.id = a.patient_id
where lower(prod.full_name) like '%подписка%' 
and a.finished > p.start_
and a.finished < p.end_
and lower(first_name) not like '%test%'
and a.good

"""

In [4]:
conn = connect_telemed()

2020-04-29 16:18:52 connect to postgres database using config file "/home/sergey/drclinics/common/../.credentials/telemed/prom.cfg"
2020-04-29 16:18:52 creating ssh tunnel to 172.16.100.19 as root...
2020-04-29 16:18:52 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 36897
2020-04-29 16:18:52 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None


In [5]:
df: pd.DataFrame = conn.query(sql_sub)

2020-04-29 16:18:52 @telemed query:
                    select 	prod.full_name product_name,
				    		prod.amount,
				    		patient.id patient_id,
				    		person.first_name,
				    		person.middle_name,
				    		person.last_name,
				    		par."name" partner,
				    		p.start_ + 3 * interval '1 hour' sub_start,
				    		p.end_ + 3 * interval '1 hour' sub_end
				    from promotion p
				    inner join product_condition pc
				       on p.product_condition_id = pc.id
				    inner join product prod
				       on prod.id = pc.product_id
				    ---
				    inner join patient_promotion pp on p.id = pp.promotion_id 
				    inner join patient on pp.patient_id = patient.id 
				    inner join person on patient.person_id = person.id 
				    ---
				    inner join partner par on p.partner_id = par.id
				    where lower(prod.full_name) like '%подписка%' 
				    and lower(first_name) not like '%test%'
                    None
2020-04-29 16:18:58 return 864 rows


In [6]:
df_app: pd.DataFrame = conn.query(sql_app)

2020-04-29 16:18:58 @telemed query:
                    select 	prod.full_name product_name,
				    		prod.amount,
				    		patient.id patient_id,
				    		person.first_name,
				    		person.middle_name,
				    		person.last_name,
				    		par."name" partner,
				    		p.start_ + 3 * interval '1 hour' sub_start,
				    		p.end_ + 3 * interval '1 hour' sub_end,
				    		a.id,
				    		a.finished + 3 * interval '1 hour' app_finished
				    from promotion p
				    inner join product_condition pc
				       on p.product_condition_id = pc.id
				    inner join product prod
				       on prod.id = pc.product_id
				    ---
				    inner join patient_promotion pp on p.id = pp.promotion_id 
				    inner join patient on pp.patient_id = patient.id 
				    inner join person on patient.person_id = person.id 
				    ---
				    inner join partner par on p.partner_id = par.id
				    ---
				    full join appointment a on patient.id = a.patient_id
				    where lower(prod.full_name) li

In [7]:
conn.close()

2020-04-29 16:19:01 @telemed: closing postgres connection...
2020-04-29 16:19:01 closing ssh tunnel to 172.16.100.19...


In [8]:
#df = pd.read_csv('par-pro_202004271813.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   product_name  864 non-null    object        
 1   amount        864 non-null    object        
 2   patient_id    864 non-null    int64         
 3   first_name    864 non-null    object        
 4   middle_name   760 non-null    object        
 5   last_name     864 non-null    object        
 6   partner       864 non-null    object        
 7   sub_start     864 non-null    datetime64[ns]
 8   sub_end       864 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 60.9+ KB


In [10]:
df.partner.unique()

array(['Доктор рядом', 'Яндекс.Еда'], dtype=object)

In [11]:
df.head()

,product_name,amount,patient_id,first_name,middle_name,last_name,partner,sub_start,sub_end
0,Подписка на 1 месяц. Личная клиника. Семейный.490,490.00,3049087,Алена,Вячеславовна,Соловьева,Доктор рядом,2020-04-04 03:00:00,2020-05-04 03:00:00
1,Подписка на 1 месяц. Личная клиника. Семейный.490,490.00,3046816,Ольга,Константиновна,Безотосова,Доктор рядом,2020-04-03 03:00:00,2020-05-03 03:00:00
2,Подписка ТМ ВВ,2400.00,1118833,Коля,None,Коля,Доктор рядом,2020-04-03 03:00:00,2020-04-10 03:00:00
3,Подписка ТМ ВВ,2400.00,1118833,Коля,None,Коля,Доктор рядом,2020-04-03 03:00:00,2020-04-10 03:00:00
4,Подписка ТМ ДР Семейная 1 мес,490.00,1118833,Коля,None,Коля,Доктор рядом,2020-04-03 03:00:00,2020-04-10 03:00:00


In [12]:
df[df['partner'] == 'Яндекс.Еда']

,product_name,amount,patient_id,first_name,middle_name,last_name,partner,sub_start,sub_end
405,Яндекс.Еда_Подписка,25.00,2583249,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
435,Яндекс.Еда_Подписка,25.00,2583219,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
436,Яндекс.Еда_Подписка,25.00,2583206,Ксения,Марсовна,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
443,Яндекс.Еда_Подписка,25.00,2583249,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
595,Яндекс.Еда_Подписка,25.00,2583219,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
599,Яндекс.Еда_Подписка,25.00,2583206,Ксения,Марсовна,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00


In [13]:
df[df['product_name'] == 'Яндекс.Еда_Подписка']

,product_name,amount,patient_id,first_name,middle_name,last_name,partner,sub_start,sub_end
405,Яндекс.Еда_Подписка,25.00,2583249,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
435,Яндекс.Еда_Подписка,25.00,2583219,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
436,Яндекс.Еда_Подписка,25.00,2583206,Ксения,Марсовна,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
443,Яндекс.Еда_Подписка,25.00,2583249,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
595,Яндекс.Еда_Подписка,25.00,2583219,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
599,Яндекс.Еда_Подписка,25.00,2583206,Ксения,Марсовна,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00


In [14]:
df.groupby(['product_name', 'partner'])[['patient_id']].count().reset_index()

,product_name,partner,patient_id
0,Годовая подписка – базовый_1800,Доктор рядом,7
1,Подписка ТМ ВВ,Доктор рядом,24
2,Подписка ТМ ВВ мес.,Доктор рядом,32
3,Подписка ТМ ВВ.Довериум мес.,Доктор рядом,18
4,Подписка ТМ ДР Семейная 1 мес,Доктор рядом,2
5,Подписка на 1 месяц. Базовый,Доктор рядом,41
6,Подписка на 1 месяц. Детский. 150,Доктор рядом,404
7,Подписка на 1 месяц. Детский. 190,Доктор рядом,96
8,Подписка на 1 месяц. Детский. 290,Доктор рядом,80
9,Подписка на 1 месяц. Детский. 390,Доктор рядом,1


In [15]:
df.groupby(['partner'])[['patient_id']].count().reset_index().rename(columns={'patient_id':'total_patients'})

,partner,total_patients
0,Доктор рядом,858
1,Яндекс.Еда,6


In [16]:
df_subscribe = pd.DataFrame(df.groupby(['partner', 'product_name'])[['patient_id']].count()).reset_index()

In [17]:
df_subscribe = df_subscribe.rename(columns={'patient_id':'subscribe'})

In [18]:
df_subscribe # всего подписок

,partner,product_name,subscribe
0,Доктор рядом,Годовая подписка – базовый_1800,7
1,Доктор рядом,Подписка ТМ ВВ,24
2,Доктор рядом,Подписка ТМ ВВ мес.,32
3,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,18
4,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,2
5,Доктор рядом,Подписка на 1 месяц. Базовый,41
6,Доктор рядом,Подписка на 1 месяц. Детский. 150,404
7,Доктор рядом,Подписка на 1 месяц. Детский. 190,96
8,Доктор рядом,Подписка на 1 месяц. Детский. 290,80
9,Доктор рядом,Подписка на 1 месяц. Детский. 390,1


In [19]:
df_temp = pd.DataFrame(df)

In [20]:
df_temp = df_temp.groupby(['partner', 'product_name'])['patient_id'].unique().reset_index()

In [21]:
df_temp

,partner,product_name,patient_id
0,Доктор рядом,Годовая подписка – базовый_1800,"[3046858, 1118833, 955694, 4386]"
1,Доктор рядом,Подписка ТМ ВВ,"[1118833, 955694, 3191625, 3514719, 1479649, 1..."
2,Доктор рядом,Подписка ТМ ВВ мес.,"[3052726, 3052586, 1045350, 3052608, 3052652, ..."
3,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,"[3484736, 77761, 3494396, 3508587, 3510464, 11..."
4,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,[1118833]
5,Доктор рядом,Подписка на 1 месяц. Базовый,"[1118833, 1479649, 1190533, 1642619, 955694, 1..."
6,Доктор рядом,Подписка на 1 месяц. Детский. 150,"[2211623, 2927792, 2579854, 2567220, 2567727, ..."
7,Доктор рядом,Подписка на 1 месяц. Детский. 190,"[2667939, 2594846, 2903976, 2916803, 2792040, ..."
8,Доктор рядом,Подписка на 1 месяц. Детский. 290,"[2973329, 3047425, 2926181, 2925899, 2916927, ..."
9,Доктор рядом,Подписка на 1 месяц. Детский. 390,[1118833]


In [22]:
df_temp['prolong'] = ''

for i in range(df_temp.shape[0]):
    df_temp['prolong'][i] = len(df_temp['patient_id'][i])

In [23]:
df_temp

,partner,product_name,patient_id,prolong
0,Доктор рядом,Годовая подписка – базовый_1800,"[3046858, 1118833, 955694, 4386]",4
1,Доктор рядом,Подписка ТМ ВВ,"[1118833, 955694, 3191625, 3514719, 1479649, 1...",7
2,Доктор рядом,Подписка ТМ ВВ мес.,"[3052726, 3052586, 1045350, 3052608, 3052652, ...",13
3,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,"[3484736, 77761, 3494396, 3508587, 3510464, 11...",9
4,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,[1118833],1
5,Доктор рядом,Подписка на 1 месяц. Базовый,"[1118833, 1479649, 1190533, 1642619, 955694, 1...",25
6,Доктор рядом,Подписка на 1 месяц. Детский. 150,"[2211623, 2927792, 2579854, 2567220, 2567727, ...",366
7,Доктор рядом,Подписка на 1 месяц. Детский. 190,"[2667939, 2594846, 2903976, 2916803, 2792040, ...",95
8,Доктор рядом,Подписка на 1 месяц. Детский. 290,"[2973329, 3047425, 2926181, 2925899, 2916927, ...",75
9,Доктор рядом,Подписка на 1 месяц. Детский. 390,[1118833],1


In [24]:
df_subscribe['unique_users'] = ''

for i in range(df_temp.shape[0]):
    df_subscribe['unique_users'][i] = len(df_temp['patient_id'][i])

In [25]:
df_subscribe['prolongation'] = df_subscribe['subscribe'] - df_subscribe['unique_users']

In [26]:
df_subscribe

,partner,product_name,subscribe,unique_users,prolongation
0,Доктор рядом,Годовая подписка – базовый_1800,7,4,3
1,Доктор рядом,Подписка ТМ ВВ,24,7,17
2,Доктор рядом,Подписка ТМ ВВ мес.,32,13,19
3,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,18,9,9
4,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,2,1,1
5,Доктор рядом,Подписка на 1 месяц. Базовый,41,25,16
6,Доктор рядом,Подписка на 1 месяц. Детский. 150,404,366,38
7,Доктор рядом,Подписка на 1 месяц. Детский. 190,96,95,1
8,Доктор рядом,Подписка на 1 месяц. Детский. 290,80,75,5
9,Доктор рядом,Подписка на 1 месяц. Детский. 390,1,1,0


In [27]:
df_sum = df.groupby(['partner', 'product_name'])[['amount']].sum().reset_index()

In [28]:
df_sum

,partner,product_name,amount
0,Доктор рядом,Годовая подписка – базовый_1800,12600.00
1,Доктор рядом,Подписка ТМ ВВ,57600.00
2,Доктор рядом,Подписка ТМ ВВ мес.,9600.00
3,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,8820.00
4,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,980.00
5,Доктор рядом,Подписка на 1 месяц. Базовый,6150.00
6,Доктор рядом,Подписка на 1 месяц. Детский. 150,60600.00
7,Доктор рядом,Подписка на 1 месяц. Детский. 190,18240.00
8,Доктор рядом,Подписка на 1 месяц. Детский. 290,23200.00
9,Доктор рядом,Подписка на 1 месяц. Детский. 390,390.00


In [29]:
df_subscribe['total_revenue'] = ''

for i in range(df_temp.shape[0]):
    df_subscribe['total_revenue'][i] = df_sum['amount'][i]
    
df_subscribe['total_revenue'] = df_subscribe['total_revenue'].astype(int)

In [30]:
df_subscribe

,partner,product_name,subscribe,unique_users,prolongation,total_revenue
0,Доктор рядом,Годовая подписка – базовый_1800,7,4,3,12600
1,Доктор рядом,Подписка ТМ ВВ,24,7,17,57600
2,Доктор рядом,Подписка ТМ ВВ мес.,32,13,19,9600
3,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,18,9,9,8820
4,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,2,1,1,980
5,Доктор рядом,Подписка на 1 месяц. Базовый,41,25,16,6150
6,Доктор рядом,Подписка на 1 месяц. Детский. 150,404,366,38,60600
7,Доктор рядом,Подписка на 1 месяц. Детский. 190,96,95,1,18240
8,Доктор рядом,Подписка на 1 месяц. Детский. 290,80,75,5,23200
9,Доктор рядом,Подписка на 1 месяц. Детский. 390,1,1,0,390


In [31]:
df_apps_temp = df_app.groupby(['partner', 'product_name'])['patient_id'].count().reset_index().rename(columns={'patient_id':'apps'})

In [32]:
df_apps_temp

,partner,product_name,apps
0,Доктор рядом,Годовая подписка – базовый_1800,23
1,Доктор рядом,Подписка ТМ ВВ,2
2,Доктор рядом,Подписка ТМ ВВ мес.,10
3,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,40
4,Доктор рядом,Подписка на 1 месяц. Базовый,34
5,Доктор рядом,Подписка на 1 месяц. Детский. 150,1618
6,Доктор рядом,Подписка на 1 месяц. Детский. 190,339
7,Доктор рядом,Подписка на 1 месяц. Детский. 290,255
8,Доктор рядом,Подписка на 1 месяц. Личная клиника. Семейный.490,144
9,Доктор рядом,Подписка на 1 месяц. Расширенный,20


In [33]:
df_subscribe['apps'] = ''

for i in range(df_subscribe.shape[0]):
    for j in range(df_apps_temp.shape[0]):
        if df_apps_temp['partner'][j] == df_subscribe['partner'][i] and df_apps_temp['product_name'][j] == df_subscribe['product_name'][i]:
            df_subscribe['apps'][i] = df_apps_temp['apps'][j]
            break
    else:
        df_subscribe['apps'][i] = 0
    

In [34]:
df_subscribe['patient-app'] = df_subscribe['apps'] / df_subscribe['unique_users']

In [35]:
df_subscribe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   partner        14 non-null     object
 1   product_name   14 non-null     object
 2   subscribe      14 non-null     int64 
 3   unique_users   14 non-null     object
 4   prolongation   14 non-null     object
 5   total_revenue  14 non-null     int64 
 6   apps           14 non-null     object
 7   patient-app    14 non-null     object
dtypes: int64(2), object(6)
memory usage: 1.0+ KB


In [36]:
df_subscribe['patient-app'] = df_subscribe['patient-app'].astype(float)
df_subscribe = df_subscribe.round({'patient-app':2})

In [37]:
df_subscribe

,partner,product_name,subscribe,unique_users,prolongation,total_revenue,apps,patient-app
0,Доктор рядом,Годовая подписка – базовый_1800,7,4,3,12600,23,5.75
1,Доктор рядом,Подписка ТМ ВВ,24,7,17,57600,2,0.29
2,Доктор рядом,Подписка ТМ ВВ мес.,32,13,19,9600,10,0.77
3,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,18,9,9,8820,40,4.44
4,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,2,1,1,980,0,0.00
5,Доктор рядом,Подписка на 1 месяц. Базовый,41,25,16,6150,34,1.36
6,Доктор рядом,Подписка на 1 месяц. Детский. 150,404,366,38,60600,1618,4.42
7,Доктор рядом,Подписка на 1 месяц. Детский. 190,96,95,1,18240,339,3.57
8,Доктор рядом,Подписка на 1 месяц. Детский. 290,80,75,5,23200,255,3.40
9,Доктор рядом,Подписка на 1 месяц. Детский. 390,1,1,0,390,0,0.00
